In [ ]:
import os

from edf.pc_utils import draw_geometry, create_o3d_points, get_plotly_fig
from edf.data import PointCloud, SE3, TargetPoseDemo, DemoSequence, DemoSeqDataset, gzip_load
from edf.preprocess import Rescale, NormalizeColor, Downsample, PointJitter, ColorJitter
from edf.agent import PickAgent, PlaceAgent

import numpy as np
import yaml
import plotly as pl
import plotly.express as ple
import open3d as o3d

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose

torch.set_printoptions(precision= 3, sci_mode=False, linewidth=120)

In [ ]:
agent_config_dir = "config/agent_config/place_agent_dev.yaml"
train_config_dir = "config/train_config/train_place_dev.yaml"
agent_param_dir = "checkpoint/mug_10_demo/place_dev"

with open(train_config_dir) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
device = 'cpu' # config['device']
unit_len = config['characteristic_length']
temperature = config['temperature']
max_epochs = config['max_epochs']
N_transform_init = config['N_transform_init']
mh_iter_init = config['mh_iter_init']
langevin_iter_init = config['langevin_iter_init']
langevin_begin_epoch = config['langevin_begin_epoch']
report_freq = config['report_freq']
init_CD_ratio = config['init_CD_ratio']
end_CD_ratio = config['end_CD_ratio']
CD_end_iter = config['CD_end_iter']
lr_se3T = config['lr_se3T']
lr_energy_fast = config['lr_energy_fast']
lr_energy_slow = config['lr_energy_slow']
lr_query_fast = config['lr_query_fast']
lr_query_slow = config['lr_query_slow']
std_theta_perturb = config['std_theta_degree_perturb'] / 180 * np.pi
std_X_perturb = config['std_X_perturb']
edf_norm_std = config['edf_norm_std']
query_anneal_end_iter = config['query_anneal_end_iter']
query_init_temp = config['query_init_temp']
query_end_temp = config['query_end_temp']
elbo_end_iter = config['elbo_end_iter']
langevin_dt = config['langevin_dt']

In [ ]:
train_logs = gzip_load(dir=agent_param_dir, filename=f"trainlog_iter_{1}.gzip")

In [ ]:
train_logs['scene_raw'] = train_logs['scene_raw'].to('cpu')
train_logs['grasp_raw'] = train_logs['grasp_raw'].to('cpu')
train_logs['scene_proc'] = train_logs['scene_proc'].to('cpu')
train_logs['grasp_proc'] = train_logs['grasp_proc'].to('cpu')

In [ ]:
scene_raw: PointCloud = train_logs['scene_raw']
grasp_raw: PointCloud = train_logs['grasp_raw']

In [ ]:
grasp_pl = grasp_raw.plotly(point_size=1.0, name="grasp")
query_points = train_logs['edf_outputs']['query_points']
query_attention = train_logs['edf_outputs']['query_attention']
query_opacity = query_attention ** 1
query_pl = PointCloud.points_to_plotly(pcd=query_points, point_size=15.0, opacity=query_opacity / query_opacity.max())

In [ ]:
fig = get_plotly_fig()
fig = fig.add_traces([grasp_pl, query_pl])
fig.show()

In [ ]:
target_pcd = PointCloud.merge(scene_raw, grasp_raw.transformed(SE3(train_logs['target_T']))[0])
sample_pcd = PointCloud.merge(scene_raw, grasp_raw.transformed(SE3(train_logs['best_neg_T']))[0])
target_pl = target_pcd.plotly(point_size=1.0)
sample_pl = sample_pcd.plotly(point_size=1.0)

In [ ]:
fig = get_plotly_fig()
fig = fig.add_traces([sample_pl])
fig.show()